In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Load the dataset
df = pd.read_csv('Churn_Modelling.csv')

# Display the first few rows and dataset info
print("First few rows of the DataFrame:\n", df.head())
print("\nDataset Info:\n", df.info())

# Check for missing values
print("\nMissing values in the dataset:\n", df.isnull().sum())

# Drop unnecessary columns (if any) like 'CustomerId', 'Surname'
df.drop(columns=['CustomerId', 'Surname'], inplace=True)

# Display class distribution
print("\nClass distribution:\n", df['Exited'].value_counts())

# Splitting data into features and target
X = df.drop('Exited', axis=1)
y = df['Exited']

# Identify categorical and numerical features
categorical_features = ['Geography', 'Gender']
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Preprocessing steps for numerical and categorical features
numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()

# Combine transformers into a single ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Creating a pipeline with preprocessing and model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X.shape[1],)))  # Input layer
model.add(Dropout(0.5))  # Dropout layer to prevent overfitting
model.add(Dense(32, activation='relu'))  # Hidden layer
model.add(Dropout(0.5))  # Dropout layer
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Create a full pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                             ('model', model)])

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the model
pipeline.fit(X_train, y_train)

# Evaluate the model
y_pred = (pipeline.predict(X_test) > 0.5).astype("int32")  # Convert probabilities to binary

# Performance metrics
print("\nAccuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Create confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Not Exited', 'Exited'], yticklabels=['Not Exited', 'Exited'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Save the model if needed
# pipeline.named_steps['model'].save('customer_churn_model.h5')

ModuleNotFoundError: No module named 'tensorflow'